In [1]:
import glob
import multiprocessing
import os
import pprint
import re
import nltk
from gensim.models import Word2Vec as w2v
from gensim.models import Word2Vec
import codecs
import sklearn.manifold
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as py
import string

C:\Users\user\Anaconda3\envs\mydeep\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [53]:
nltk.download ('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
book_filenames=sorted(glob.glob(r"C:\Users\user\Desktop\mypre\ex3/*.txt"))
print (book_filenames)

['C:\\Users\\user\\Desktop\\mypre\\ex3\\2 - Copy.txt', 'C:\\Users\\user\\Desktop\\mypre\\ex3\\2.txt', 'C:\\Users\\user\\Desktop\\mypre\\ex3\\ሓረስታይ ወረዳ ላዕላይ ማይጨው ድሌት ተጠቓምነት መስኖ እናዓበየ ይመፅእ ከምዘሎ ቤት ፅሕፈት ምምሕዳር እታ ወረዳ ኣፍሊጡ፡፡.txt']


In [23]:
corpus_raw= u" "
for book_filename in book_filenames:
    #print (book_filename)
    print ("reading '{0}'....".format(book_filename))
    with codecs.open (book_filename,"r","utf-8") as book_file:
        #print (book_file)
        corpus_raw+=book_file.read()
        print (type(corpus_raw))
        print ("corpus_raw {0} characters long".format (len(corpus_raw)))

reading 'C:\Users\user\Desktop\mypre\ex3\2 - Copy.txt'....
<class 'str'>
corpus_raw 1249 characters long
reading 'C:\Users\user\Desktop\mypre\ex3\2.txt'....
<class 'str'>
corpus_raw 2497 characters long
reading 'C:\Users\user\Desktop\mypre\ex3\ሓረስታይ ወረዳ ላዕላይ ማይጨው ድሌት ተጠቓምነት መስኖ እናዓበየ ይመፅእ ከምዘሎ ቤት ፅሕፈት ምምሕዳር እታ ወረዳ ኣፍሊጡ፡፡.txt'....
<class 'str'>
corpus_raw 3134 characters long


In [24]:
tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

In [25]:
print (tokenizer)

In [11]:
raw_sentences=tokenizer.tokenize(corpus_raw)

In [21]:
print (raw_sentences[0])

  ህዝቢ ወረዳ ላዕለይ ማይጨው ምስ ኣካላት ምምሕዳር ብምርድዳእ ኣብ ዝቅፅሉ 5 ዓመታት ዝፍፀሙ መደባት ብምዕዋት ትልሚ ዕብየትን ትራንስፎርሜሽንን 5 ዓመት ኣጠናኪሮም ከምዝቕፅሉ ሓቢሮም፡፡


In [15]:
def sentences_to_wordlist(raw):
    clean = re.sub("[+:\.\!\/_,$%^*(+\"\'<>=]+|[+——！，。？、~@#￥%……&*（）]+", " ",raw)
   # print (clean)
    words = clean.split()
   # print (words)
    return words

In [16]:
sentences=[]
for raw_sentence in raw_sentences:
   # print (raw_sentence)
    if len(raw_sentence)>0:
        sentences.append(sentences_to_wordlist(raw_sentence))

In [17]:
print (raw_sentences[0])
print (sentences_to_wordlist(raw_sentences[0]))


  ህዝቢ ወረዳ ላዕለይ ማይጨው ምስ ኣካላት ምምሕዳር ብምርድዳእ ኣብ ዝቅፅሉ 5 ዓመታት ዝፍፀሙ መደባት ብምዕዋት ትልሚ ዕብየትን ትራንስፎርሜሽንን 5 ዓመት ኣጠናኪሮም ከምዝቕፅሉ ሓቢሮም፡፡
['\ufeffኣብ', 'ዞባ', 'ደቡብ', 'ምብራቅ', 'ወረዳ', 'ሕንጣሎ', 'ወጀራት', 'ጣብያ', 'ዋዛ', 'ዓዲኣዋና', 'እትርከብ', 'ቤት', 'ትምህርቲ', 'ዋዛ', 'ካብ', '1ይ', 'ክሳብ', '8ይ', 'እተምህር', 'ቤት', 'ትምህርቲ', 'እንትትኸውን', 'ህዝቢ', 'እታ', 'ጣብያ', 'ምስ', 'ተምሃሮን', 'መምህረን', 'ተሓባቢሮም', 'ብምስራሕ', 'ፅሬት', 'ትምህርቲ', 'ኣረጋጊፆም', 'ኣብቲ', 'ጣብያ', 'ጥሙር', 'ጥዕና', 'ፓኬጅ', 'እተኣታቶ', 'ስነ', 'ዘራእቲ', 'ኣግራብን', 'ክሕለውን', 'ዕዙዝ', 'ስራሕቲ', 'ብምስራሕ', 'ተሸላሚት', 'ሞዴል', 'ኾይና', 'እያ፡፡', 'ሓላፊት', 'ምክትል', 'ክፍሊ', 'ትምህርቲ', 'ወረዳ', 'ሕንጣሎ', 'ወጀራት', 'ወ', 'ሮ', 'ምዕራፍ', 'ብዛዕባ', 'ቤት', 'ትምህርቲ', 'ዋዛ', 'እንትገልፃ', 'እዚ', 'ቤት', 'ትምህርቲ', 'እዚ', 'ብምእታዩ', 'እዩ', 'ኣተሓሒዘን', 'ቢሮ', 'ትምህርቲ', '53', 'መዓቐኒ', 'ሞዴል', 'ኣብያተ', 'ትምህርቲ', 'ኣውፂኡ', 'በዚ', 'ዝተዓቐና', '72', 'ኣብያተ', 'ትምህርቲ', 'እቲ', 'ወረዳ', 'እተን', '29', 'ሞዴል', 'እንትኾና', '34', 'ሓውሲ', 'ሞዴል', '11', 'ቤት', 'ትምህርቲ', 'ድሕረት', 'ዝተሰርዓ', 'እንትኾና', 'ቤት', 'ትምህርቲ', 'ዋዛ', 'ካብተን', '29', 'ቅድሚት', 'ዝስረዓ', 'ሞዴል', 'ክኸውን', 'ዝከኣላ', 'ተሳትፎ', 'ሕብረተሰብ', 'ናተይነት', 'ተሰሚዕዎ', 'ተሓባቢሩ', 

In [26]:
token_count=sum([len(sentence)for sentence in sentences])
print (token_count)
print ("the book corp {0:} contains tokens".format(token_count))

660
the book corp 660 contains tokens


In [63]:
num_features=300
min_word_counts=3
num_workers=multiprocessing.cpu_count()
context_size=7
#downsampling=1e-3
#seed=1
#my_rule=3

In [64]:
mycorpus2vec=Word2Vec(sg=1,workers=num_workers,min_count=min_word_counts,window=context_size)
#print (sentences)

In [65]:
mycorpus2vec.build_vocab(sentences)

In [66]:
print ("word2vec vocuablary length", len(mycorpus2vec.wv.vocab))

word2vec vocuablary length 42


In [67]:
mycorpus2vec.train(sentences,total_examples=mycorpus2vec.corpus_count,epochs=mycorpus2vec.iter)

309

In [68]:
if not os.path.exists("trained"):
    os.mkdirs("trained")

In [72]:
mycorpus2vec=mycorpus2vec.save(os.path.join("trained","mycorpus2vec.train"))
#mycorpus2vec.save_word2vec_format(r"C:\Users\use\mycorpus2vec.train.bin",binary=True)

In [70]:
#mycorpus2vec.save("mycorpus2vec.train")

In [66]:
#model = mycorpus2vec.load_word2vec_format(r'C:\Users\use\mycorpus2vec.train.bin', binary=True)

In [80]:
#Word2Vec.save_word2vec_format(r"C:\Users\use\mycorpus2vec.bin",binary=True)
mycorpus2vec.save_word2vec_format(r"C:\Users\user\trained\mycorpus2vec2.train",binary=True)


AttributeError: 'NoneType' object has no attribute 'save_word2vec_format'